# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re

import pickle

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

%load_ext line_profiler

[nltk_data] Downloading package punkt to /home/martinp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/martinp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/martinp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
with engine.connect() as connection:
    df = pd.read_sql_table(table_name='messages', con=connection)

df.replace(2, 1, inplace=True)

X = df.loc[:, 'message']
y = df.iloc[:, 4:]

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### 2. Write a tokenization function to process your text data

In [4]:
y.sum()

related                   20094
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire                        282
earthqua

In [5]:
def tokenize(text):
    text = re.sub(r"[^A-Za-z0-9\s]", " ", text)
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    words = word_tokenize(text.lower().strip())
    words = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stopwords.words("english")]
    
    return words

In [6]:
tokenize(X[355])

['town',
 'outside',
 'zone',
 'called',
 'dezam',
 'komin',
 'veret',
 'artibonite',
 'hit',
 'morning',
 'earthquake',
 'house',
 'crumbled',
 'please',
 'help']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state=42)

In [9]:
%%time

#%lprun -f tokenize 
pipeline.fit(X_train, y_train)

CPU times: user 9.02 s, sys: 700 ms, total: 9.72 s
Wall time: 9.74 s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbde8c39f70>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=LinearSVC()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
%%time
pipeline.score(X_test, y_test)

CPU times: user 34.7 s, sys: 3.04 s, total: 37.7 s
Wall time: 37.8 s


0.2598579125542364

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y_test.columns.astype('str')

In [13]:
# colname = 'related'
# print(classification_report(y_test[colname], y_pred_df[colname]))

In [14]:
#for column in y_test.columns.astype('str'):
#    print(classification_report(y_test[column], y_pred_df[column]))

In [15]:
print(classification_report(y_test, y_pred_df, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.88     16033
               request       0.74      0.56      0.63      3595
                 offer       0.00      0.00      0.00        95
           aid_related       0.70      0.66      0.68      8665
          medical_help       0.57      0.21      0.30      1667
      medical_products       0.73      0.21      0.32      1057
     search_and_rescue       0.72      0.10      0.17       576
              security       0.50      0.00      0.01       359
              military       0.58      0.23      0.33       664
                 water       0.74      0.58      0.65      1333
                  food       0.81      0.67      0.73      2316
               shelter       0.73      0.48      0.58      1835
              clothing       0.75      0.28      0.41       321
                 money       0.70      0.12      0.21       472
        missing_people       0.71      

/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fbde8c39f70>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=LinearSVC()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fbde8c39f70>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LinearSVC()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__es

In [17]:
parameters = {
    'vect__tokenizer': [None, tokenize],
    'clf__estimator__multi_class': ['ovr', 'crammer_singer'],
    'clf__estimator__max_iter': [2000, 8000, 15000],
    'clf__estimator__dual':[False, True]
}
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=True)

In [18]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fbde8c39f70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LinearSVC()))]),
             param_grid={'clf__estimator__dual': [False, True],
                         'clf__estimator__max_iter': [2000, 8000, 15000],
                         'clf__estimator__multi_class': ['ovr',
                                                         'crammer_singer'],
                         'vect__tokenizer': [None,
                                             <function tokenize at 0x7fbde8c39f70>]},
             verbose=True)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.best_params_

{'clf__estimator__dual': False,
 'clf__estimator__max_iter': 2000,
 'clf__estimator__multi_class': 'crammer_singer',
 'vect__tokenizer': <function __main__.tokenize(text)>}

In [20]:
cv.score(X_test, y_test)

0.2607638392218567

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
pipeline8 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [22]:
pipeline8.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbde8c39f70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [23]:
pipeline8_pred_test_df = pd.DataFrame(pipeline8.predict(X_test))

In [24]:
print(classification_report(y_test, pipeline8_pred_test_df, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88     16033
               request       0.83      0.46      0.59      3595
                 offer       0.00      0.00      0.00        95
           aid_related       0.74      0.66      0.70      8665
          medical_help       0.59      0.05      0.09      1667
      medical_products       0.89      0.04      0.07      1057
     search_and_rescue       1.00      0.01      0.03       576
              security       0.00      0.00      0.00       359
              military       0.80      0.06      0.11       664
                 water       0.89      0.32      0.46      1333
                  food       0.84      0.55      0.66      2316
               shelter       0.85      0.34      0.48      1835
              clothing       0.80      0.02      0.05       321
                 money       0.75      0.01      0.03       472
        missing_people       1.00      

/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinp/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [25]:
with open('message-classifier.pkl', 'wb') as message_classifier_file:
    pickle.dump(cv.estimator, message_classifier_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.